In [2]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2



In [3]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear3 = nn.Linear(128, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = self.linear3(output)
        distribution = F.softmax(output, dim=-1)
        return distribution


class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear3 = nn.Linear(128, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        # output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

In [16]:
env = gym.make("MountainCar-v0")


In [17]:
gamma = .99
num_episodes = 5000
num_steps = 1000
env = gym.make("MountainCar-v0")
# env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

actor = Actor(state_size, action_size)
critic = Critic(state_size, action_size)

count_episode = range(1,num_episodes+1)
count_actions = []
total_count_actions = []
total_a = 0
for episode in range(num_episodes):
    actor_optim = optim.Adam(actor.parameters(),lr=0.001)
    critic_optim = optim.Adam(critic.parameters(), lr=0.001)
    state = env.reset()
    isTerminal = False
    score = 0
    
    count_a = 0
    
    for i in range(num_steps):       
        count_a += 1
        state = torch.FloatTensor(state)
        with torch.no_grad():
            action = torch.argmax(actor(state))
            log_prob = torch.max(actor(state))
        state_prime, reward, isTerminal, info = env.step(action.item())
        state_prime = torch.FloatTensor(state_prime)
        if state_prime[0] >= 0.5:
            print(f'Num episodes {episode}, num actions {i} {isTerminal}')
            v_next = torch.tensor([0]).float().unsqueeze(0)
        # if isTerminal:
        #     print(f'Num episodes {episode}, num actions {i} {isTerminal}')
        v_curr = critic(state)
        v_next = critic(state_prime)
            

        td_target = reward + gamma * v_next
        td_error = reward + ((gamma*v_next)-v_curr)
        
        # print(v_curr)
        # print(log_prob)
        # Policy
        actor_loss = -(td_error)
        actor_loss *= log_prob
        actor_optim.zero_grad()
        actor_loss.backward(retain_graph=True)
        actor_optim.step()

        # Value
        critic_loss = F.mse_loss(td_target,v_curr)
        critic_optim.zero_grad()
        critic_loss.backward()
        critic_optim.step()
        state = state_prime

        # print(f'Actor loss is {actor_loss} and critic loss is {critic_loss}')
        if state_prime[0] >= 0.5:
            break
        # print(state)
    # print(count_a)
    count_actions.append(count_a)
    total_a += count_a
    total_count_actions.append(total_a)
      
torch.save(actor, 'actor.pkl')
torch.save(critic, 'critic.pkl')
env.close()        

plt.figure()
plt.title('Count of Episodes vs Count of Actions')
plt.xlabel('Count of Episodes')
plt.ylabel('Count of Actions')
plt.plot(count_episode, count_actions)
plt.savefig('count_actions_ac.jpg')
plt.show()

plt.figure()
plt.title('Total Actions vs Count of Episodes ')
plt.ylabel('Count of Episodes')
plt.xlabel('Total Count of Actions')
plt.plot(total_count_actions, count_episode)
plt.savefig('total_actions_ac.jpg')
plt.show()

In [16]:
state

tensor([-0.3171,  0.0036])

In [6]:
import gym
env = gym.make('MountainCar-v0')
env.reset()

for i in range(1000):
    action = torch.argmax(actor(state))
    state_prime, reward, isTerminal, info = env.step(action.item())
    state_prime = torch.FloatTensor(state_prime)
    state = state_prime
    # env.step(env.action_space.sample())
    env.render()